# Notebook Imports

In [64]:
import pandas as pd
import numpy as np

# Constants

In [65]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data-s.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data-s.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

# Read and Load Features from .txt Files into NumPy Array

In [66]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [67]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [68]:
sparse_train_data[:5]

array([[ 0,  0,  1,  2],
       [ 0,  5,  1,  1],
       [ 0, 10,  1,  2],
       [ 0, 47,  1,  1],
       [ 0, 75,  1,  1]])

In [69]:
sparse_train_data[-5:]

array([[5798, 1887,    0,    1],
       [5798, 2152,    0,    1],
       [5798, 2206,    0,    1],
       [5798, 2392,    0,    1],
       [5798, 2487,    0,    1]])

In [70]:
print('Nr of rows in training file', sparse_train_data.shape[0])
print('Nr of rows in test file', sparse_test_data.shape[0])

Nr of rows in training file 266236
Nr of rows in test file 109917


In [71]:
print('Nr of emails in training file', np.unique(sparse_train_data[:, 0]).size)

Nr of emails in training file 4016


In [72]:
print('Nr of emails in test file', np.unique(sparse_test_data[:, 0]).size)

Nr of emails in test file 1721


### How to create an Empty DataFrame

In [73]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [74]:
len(column_names)

2502

In [75]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5796, 5797, 5798])

In [76]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [77]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
sparse_train_data[10:13]

array([[   0,  568,    1,    1],
       [   0,  852,    1,    1],
       [   0, 1010,    1,    1]])

In [79]:
sparse_train_data[10][3]

1

# Create a Full Matrix from a Sparse Matrix

In [80]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe. 
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens. 
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix
    

In [81]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

CPU times: user 7.82 s, sys: 87.6 ms, total: 7.91 s
Wall time: 7.89 s


In [82]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5792,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,2,0,1,1,1,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5796,0,1,0,4,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5797,0,1,0,2,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5798,0,1,2,1,0,2,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model

## Calculating the Probability of Spam

In [83]:
# Challenge: Calculate the probability of spam - the percent of spam messages in the training
# dataset. Store this value in a variable called prob_spam

In [84]:
full_train_data.CATEGORY.size

4016

In [85]:
full_train_data.CATEGORY.sum()

1250

In [86]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is', prob_spam)

Probability of spam is 0.3112549800796813


## Total Number of Words / Tokens

In [87]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,2,1,0,3,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,2,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,3,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [88]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4016,)

In [89]:
email_lengths[:5]

DOC_ID
0     22
1     56
2    234
3     37
4    260
dtype: int64

In [90]:
total_wc = email_lengths.sum()
total_wc

445901

## Number of Tokens in Spam & Ham Emails

**Challenge** Create a subset of the email_lengths series that only contains the spam messages. Call the subset spam_lengths. Then count the total number of words that occur in spam emails. 

Do the same for the non-spam emails. Create a subset called ham_lengths. Then count the total number of words that occur in the ham emails. 

In [91]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1250,)

In [92]:
spam_wc = spam_lengths.sum()
spam_wc

185054

In [93]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2766,)

In [94]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [95]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

260847

In [96]:
spam_wc + nonspam_wc - total_wc

0

In [97]:
# Challenge: Can you figure out if spam emails or non-spam emails tend to be longer? Which
# category contains more tokens? Take a guess and then verify your intuition using the dataset

In [98]:
print('Average nr of words in spam emails {:.0f}'.format(spam_wc / spam_lengths.shape[0]))
print('Average nr of words in ham emails {:.3f}'.format(nonspam_wc / ham_lengths.shape[0]))

Average nr of words in spam emails 148
Average nr of words in ham emails 94.305


## Summing the Tokens Occuring in Spam

In [99]:
full_train_features.shape

(4016, 2500)

In [100]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,2,1,0,3,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,2,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,3,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [101]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1887,0,2,3,1,0,3,2,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1889,3,0,1,1,2,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1890,2,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,0,0,1,0,2,1,2,8,2,0,...,0,0,0,0,0,0,0,0,0,0
1896,0,2,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [102]:
train_spam_tokens.shape

(1250, 2500)

In [103]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [104]:
summed_spam_tokens.tail()

2495    19
2496     2
2497     3
2498    26
2499    11
dtype: int64

In [105]:
# Challenge: Repeat this process for the ham messages. Sum the tokens that occur in the 
# non-spam messages. Store the values in a variable called summed_ham_tokens. 

## Summing the Tokens Occuring in Ham

In [106]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [107]:
summed_ham_tokens.shape

(2500,)

In [108]:
summed_ham_tokens.tail()

2495    23
2496    29
2497    29
2498     7
2499    26
dtype: int64

In [109]:
train_ham_tokens[2499].sum() + 1

26

## P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [110]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.012082
1    0.004921
2    0.007326
3    0.010786
4    0.006814
dtype: float64

In [111]:
prob_tokens_spam.sum()

1.0

## P(Token | Ham) - Probability that a Token Occurs given the Email is Nonspam

In [112]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

1.0

# P(Token) - Probability that Token Occurs 

In [113]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc

In [114]:
prob_tokens_all.sum()

1.0

# Save the Trained Model

In [115]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

**Challenge:** Create a full matrix from the sparse_test_data. Time the function call. How long does it take? Separate the features and the target values. Save these as separate .txt files: a TEST_TARGET_FILE and a TEST_FEATURE_MATRIX file. 

In [116]:
sparse_test_data.shape

(109917, 4)

In [117]:
%%time
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

CPU times: user 3.59 s, sys: 43.7 ms, total: 3.63 s
Wall time: 3.63 s


In [118]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [119]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)